In [ ]:
import pyodbc
import pandas as pd

# Database connection parameters
server = 'wrt-sql-prod'
database = 'dvrtDB'
username = 'wrtsqlq'
password = 'guest'

# Establish connection and fetch data
with pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password}"
) as conn:
    query = "SELECT * FROM dbo.DAILY_RECORDS WHERE STATION_ID = 10987"
    df = pd.read_sql_query(query, conn)

# Extract day columns dynamically
day_columns = [col for col in df.columns if col.startswith('RV_')]

# Create a list to hold processed rows
processed_rows = []

# Iterate over rows and day columns to build a structured DataFrame
for _, row in df.iterrows():
    year = row['RECORD_YEAR']
    for day_col in day_columns:
        day_str = day_col[3:]  # Extract day string like '0101'
        try:
            date = pd.to_datetime(f"{year}{day_str}", format='%Y%m%d')
            discharge = row[day_col]  # Keep original value, including NaN/NULL
            processed_rows.append({'DATE': date, 'DISCHARGE': discharge})
        except ValueError:
            processed_rows.append({'DATE': None, 'DISCHARGE': row[day_col]})  # Keep discharge even if date is invalid

# Create a new DataFrame
processed_df = pd.DataFrame(processed_rows)

# Export to CSV
#processed_df.to_csv('daily_discharge_data.csv', index=False)